In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa
import warnings
warnings.filterwarnings("ignore")
import wave

import json

In [2]:
path = './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation'
wav_lists = glob(path + './원천데이터/라디오/*/*.wav')
json_lists = glob(path + './라벨링데이터/라디오/*/*.json')

In [3]:
json_lists

['./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\강의_강연\\AHABM21000380.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\강의_강연\\AHABM21000381.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000001.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000002.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000003.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000004.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000005.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000006.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000007.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000008.json',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./라벨링데이터/라디오\\건강_의학\\SLABQ21000009.json',
 './001.방송 콘텐츠 대화체 음성

In [4]:
# json 파일명, start,end값 가져오기
json_file=[]
json_start=[]
json_end=[]

for json_list in json_lists:
    with open(json_list,'r',encoding="UTF-8") as f:
        json_data = json.load(f)
#         print(json_data['utterance'])
        txts = json_data['utterance']
        for txt in txts:
            if txt['speaker_role'] == '진행자':
                json_file.append(txt['id'].split('.')[0])
                json_start.append(txt['start'])
                json_end.append(txt['end'])

In [5]:
# json_df 생성
json_df = pd.DataFrame([json_file, json_start,json_end])
json_df = json_df.T
json_df.columns = ['file_name','start','end']
json_df.value_counts()

file_name      start     end     
AHABK21001700  0.000     2.901       1
SLABN21000022  1308.503  1330.919    1
               1174.738  1187.880    1
               1187.880  1189.313    1
               1190.055  1192.571    1
                                    ..
SLABC21000045  1356.291  1360.460    1
               1350.341  1356.291    1
               1302.171  1306.237    1
               1291.531  1302.171    1
SLABV21000022  997.410   997.904     1
Length: 146332, dtype: int64

In [6]:
json_df[json_df['file_name'] == 'SLABS21000017']
set(json_file)

{'AHABK21001700',
 'AHABK21001701',
 'AHABK21001702',
 'AHABK21001703',
 'AHABM21000380',
 'AHABM21000381',
 'DGABO21000007',
 'SLABA21000001',
 'SLABA21000002',
 'SLABA21000003',
 'SLABA21000004',
 'SLABA21000005',
 'SLABA21000006',
 'SLABA21000007',
 'SLABA21000008',
 'SLABA21000009',
 'SLABA21000010',
 'SLABA21000011',
 'SLABA21000012',
 'SLABA21000013',
 'SLABA21000014',
 'SLABA21000015',
 'SLABA21000016',
 'SLABA21000017',
 'SLABA21000018',
 'SLABA21000019',
 'SLABA21000020',
 'SLABA21000021',
 'SLABA21000022',
 'SLABA21000023',
 'SLABA21000024',
 'SLABA21000025',
 'SLABA21000026',
 'SLABA21000027',
 'SLABA21000028',
 'SLABA21000029',
 'SLABA21000030',
 'SLABA21000031',
 'SLABA21000032',
 'SLABA21000033',
 'SLABA21000034',
 'SLABA21000035',
 'SLABA21000036',
 'SLABA21000037',
 'SLABA21000038',
 'SLABA21000039',
 'SLABA21000040',
 'SLABA21000041',
 'SLABA21000042',
 'SLABA21000043',
 'SLABA21000044',
 'SLABA21000045',
 'SLABA21000046',
 'SLABA21000047',
 'SLABA21000048',
 'SLABA210

In [7]:
# WAV파일에서 파일명 추출
file_name = []
for wav_list in wav_lists:
    name = wav_list.split('\\')[-1][:-4]
    file_name.append(name)
    

# wav 데이터 프레임 생성
wav_df = pd.DataFrame([file_name,wav_lists])
wav_df = wav_df.T
wav_df.columns = ['file_name','wav_path']
wav_df

,file_name,wav_path
0,AHABM21000380,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
1,AHABM21000380.wav_v1,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2,AHABM21000380_v3,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
3,AHABM21000380_v4,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
4,AHABM21000381,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
5,AHABM21000381.wav_v1,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
6,AHABM21000381_v3,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
7,AHABM21000381_v4,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
8,AHABM2100038_v2,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
9,SLABQ21000001,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....


In [8]:
# json 데이터프레임& wav 데이터프레임 합치기
df = pd.merge(json_df, wav_df, on='file_name', how='outer')
df1 = df.copy().dropna()

df1['start'] = df1['start'].astype(float)
df1['end'] = df1['end'].astype(float)

df1

,file_name,start,end,wav_path
0,AHABM21000380,0.000,5.796,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
1,AHABM21000380,5.796,9.141,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2,AHABM21000380,9.141,11.360,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
3,AHABM21000380,11.360,16.177,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
4,AHABM21000380,16.177,21.251,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
...,...,...,...,...
2447,SLABQ21000015,1813.123,1813.652,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2448,SLABQ21000015,1815.954,1823.795,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2449,SLABQ21000015,1824.623,1825.981,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2450,SLABQ21000015,1826.866,1828.138,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....


In [9]:
def load_data(paths):
    result = []
    for path in tqdm(paths):
        # sr = 16000이 의미는 1초당 16000개의 데이터를 샘플링
        data, sr = librosa.load(path, sr = 16000)
        result.append(data)
    result = np.array(result) 
    # 메모리가 부족할 때는 데이터 타입을 변경 ex) np.array(data, dtype = np.float32)
    return result

In [10]:
def extract_sound(paths):
    result = []
    for path in tqdm(paths):
        # sr = 16000이 의미는 1초당 16000개의 데이터를 샘플링
        data, sr = librosa.load(path, sr = 16000)
        data = data
        result.append(data)
    result =np.array(result)
    return result

In [11]:
df1

,file_name,start,end,wav_path
0,AHABM21000380,0.000,5.796,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
1,AHABM21000380,5.796,9.141,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2,AHABM21000380,9.141,11.360,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
3,AHABM21000380,11.360,16.177,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
4,AHABM21000380,16.177,21.251,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
...,...,...,...,...
2447,SLABQ21000015,1813.123,1813.652,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2448,SLABQ21000015,1815.954,1823.795,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2449,SLABQ21000015,1824.623,1825.981,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....
2450,SLABQ21000015,1826.866,1828.138,./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation....


In [26]:
import wave

# times between which to extract the wave from

for i in range(len(df1)):
    result = []
    path = df1['wav_path'][i][:-4]
    start = int(df1['start'][i]) # seconds
    end = int(df1['end'][i])# seconds

    # file to extract the snippet from
    with wave.open(f'{path}.wav', "rb") as infile:
        # get file data
        nchannels = infile.getnchannels()
        sampwidth = infile.getsampwidth()
        framerate = infile.getframerate()
        # set position in wave to start of segment
        infile.setpos(int(start * framerate))
        # extract data
        data = infile.readframes(int((end - start) * framerate))

    # write the extracted data to a new file
    with wave.open(f'{path}_v6.wav', 'w') as outfile:
        outfile.setnchannels(nchannels)
        outfile.setsampwidth(sampwidth)
        outfile.setframerate(framerate)
        outfile.setnframes(int(len(data) / sampwidth))
        outfile.writeframes(data)

In [22]:
import soundfile as sf

def split_wav(df1):
    for i in range(len(df1)):
        result = []
        path = df1['wav_path'][i]
        data, sr = librosa.load(path, sr=16000)
        start = int(df1['start'][i])*16000
        end = int(df1['end'][i])*16000
#         print(start,end)

        result = (data[start:end]
        
        file_name1 = path[:-4]
        
#         librosa.output.soundfile.write(f'{df1["wav_path"]}.wav', data, sr)
#         sf.write(f'{file_name1}_v5.wav', result, sr, format='WAV', endian='LITTLE', subtype='PCM_16') # 깨지지 않음

    return result
   

In [ ]:
# for temp_data in tqdm(data):
#     split_wav(temp_data, df1)

In [ ]:
# for i in wav_lists:
#     if os.path.exists(i):
#         os.remove(i)

In [ ]:
# dir(librosa)

In [23]:
split_wav(df1)

RuntimeError: Error opening './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\강의_강연\\AHABM21000380_v5.wav': Format not recognised.

In [27]:
path = './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation'

split_file = glob(path + './원천데이터/라디오/*/*_v6.wav')
split_file
                

['./001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\강의_강연\\AHABM21000380_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\강의_강연\\AHABM21000381_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000001_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000002_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000003_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000004_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000005_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000006_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000007_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000008_v6.wav',
 './001.방송 콘텐츠 대화체 음성인식 데이터/01.데이터/2.Validation./원천데이터/라디오\\건강_의학\\SLABQ21000009_v6.wav',
 './001.방송